# Write your first ML model

Whether or not this is actually your first model, we are going to take a look at the
famous MNIST data set of handwritten digits. We are going to use a few different ML libs for this.

## Notebook basics
- Use Enter and Escape to toggle in and out of cells.
- When you are out you can use 'a' and 'b' to insert a new cell above or below.
- Or you can delete a cell with 'dd'.
- You can run the current cell with shift enter (works in both modes).
- You may prefer alt enter which runs cell and creates a new cell after.

## H2O
Let's start our journey by launching a local H2O cluster with one node.

In [ ]:
import h2o
h2o.init()

## Cheats
Run the cell below to enable skeletons, hints and cheats.
To later use these run hint('id') or cheat('id') in any cell.

For example hint('c') will give the first hint for exercise (c).

In [ ]:
from cheat import *

### Exercise (a)

* Take a quick look at the output from h2o.init() above. Anything particularly interesting?

### Exercise (b)

It is time to import some data into our H2O cluster. There are two data files in
the notebook work directory. Either use jupyter browser to find the files or run the cell below.

- Find them and import them into H2O. (use hint if you have trouble finding the method name.)
- View your result X by putting X last in a cell and run the cell.
- Run X.show() and display(X).
- See if you can view X in the Flow UI.

In [ ]:
!ls

In [ ]:
test = __fill_in__
train = __fill_in__

### Exercise (c)
* Find the target column of the two imported frames and convert its datatype to enum. The conversion is done by the 'asfactor' method. To select a column from a dataframe use square brackets df\[...\]. What goes between brackets can be a column name, number, or a list.

In [ ]:
train[fill_in] = train[fill_in].asfactor()
test[fill_in] = test[fill_in].asfactor()

### Exercise (d)

Often one will split off a part of training data into a validation set.
The validation set serves as a proxy for testing models 'in the wild'.
Scores measured on the training set are not reliable as many algorithms produce
perfect or near perfect training scores.
* Split the training frame into two parts: train (80% of rows) and valid (remaining 20%).

In [ ]:
# Your code here #

display(valid.head(3))
display(train.head(3))

## Ploting images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# converts to pandas dataframe (if pandas is installed)
test_pd = test.as_data_frame()
pixels = test_pd.iloc[:,list(range(0,28*28))]
labels = test_pd.iloc[:,28*28]

def draw_pixels(pixels, labels, num):
    data = np.array(pixels.iloc[num], dtype='uint8')
    data = data.reshape((28, 28))
    plt.title('Label is {label}'.format(label=labels.iloc[num]))
    plt.imshow(data, cmap='gray')
    plt.show()
    
draw_pixels(pixels, labels, 0)

## Finally Machine Learning

### Exercise (e)
Python is object oriented and objects can be created as follows:
Let's say we want an instance of MyClass. MyClass will also act as a method so MyClass() will return a new instance. As any other python method it can have a number of required parameters, followed by optional parameters with default values. For example:

obj = AnotherClass(param1 , param2, named_param1=value1, named_param2=value2)

- Create a random forest model using H2O with default settings for everything.

In [ ]:
from h2o.estimators import H2ORandomForestEstimator
rf = __fill_in__

### Exercise (f)
- Find the index of the target column using the cell below.
- Train the model. Run help(rf.train) and read about the needed parameters. Use parameters x,y,training_frame and validation_frame, but skip the rest.

Things to look at:
- While your model trains, go into FLOW and click Model -> List all models -> Click your model. Finally you can click refresh to see stats update continuously.
- Take a glance at logloss, variable importance, hitratio (see below), and more. How do these differ between training set and validation set?

In [ ]:
# replace 13 by index of target column
# for column 13 we see grayscale values 0-255, but target column should have values 0-9
train[13].unique().sort(by = 0)

In [ ]:
# this is for creating a list of consecutive integers
print(range(5))
print(list(range(5)))

In [ ]:
x = list(range(__fill_in_number_of_feature_columns__))
y = __fill_in_index_of_target_column__

In [ ]:
# test that target column is correct
assert train[y].unique().nrows==10

In [ ]:
rf.train(__fill_in_parameters__)

## Test performance

### Exercise (g)
Using rf.model_performance(df) lets you measure various performance metrics on a data frame df without going into the FLOW ui.
* Check the random forest's performance on train, validation and test sets.

## Deep Learning
I mean why not...

### Exercise
Repeat the procedure above but change H2ORandomForestEstimator to H2ODeepLearningEstimator.

- Which is better out of the box for this data set?
- Which took longer to train?
- How was the gap between validation and test errors?

## Downloading POJO/MOJO
H2O models can be downloaded in two Java compatible forms. 

## Goodbye H2O

In [ ]:
h2o.cluster().shutdown()

### Keras on Tensorflow
So far we have not exploited at all the position of pixels in the plane. It is widely known that convolutional networks (CNNs) are a great way share weights among translated descriptors. This is a fancy way of saying that we can keep the number of variables (weights) much smaller by encoding assumptions about images in the structure of the model. For example, if we have a 5x5 filter that can detect horizontal edges in one part of the image, we have reason to believe it has the same ability applied in a different part of the image.

TensorFlow is a Google library for graph computations (and neural networks as a special but important case). TensorFlow is popular also because it provides a general backend so that any or most neural networks can be implemented on top of it and it is possible to export models and run them in a JVM.

Keras is a high level library that can use TensorFlow as a backend, but makes life much easier for the programmer.

Hence we will use Keras-TF to train a CNN.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau

### Loading data as pandas Dataframe
Pandas dataframes are similar to H2O's, but run in a single python instance*. Compared to H2O's dataframes,
pandas offers far more operations and seems to be more mature as of today. By default pandas will try
to guess the schema of csv files by parsing the first row. As our data does not have a header, we have to specify that.

*The DASK project provides dataframes distributed on a cluster which are backed by pandas dataframes.

In [ ]:
train = pd.read_csv('data/mnist/mnist.train.csv.gz', header=None)
test = pd.read_csv('data/mnist/mnist.test.csv.gz', header=None)

### Exercise (h)
- Create dataframes X_train and X_test which consist of all columns except the target column. Run help(pd.DataFrame.drop) and find out what arguments you need to provide.
- Create Series Y_train and Y_test which consist of the target column only. Run help(pd.DataFrame.loc) and read the first example and the one on 'Set value for an entire column' to get a feel for slicing and indexing. However iloc is more convenient here. (There is also help on iloc but not as extensive.)
- Delete train and test to save memory.

In [ ]:
Y_train = train.iloc[fill_in_to_select_all_rows,fill_in_to_select_target_column]
X_train = train.drop(fill_in)
Y_test = test.iloc[fill_in,fill_in]
X_test = test.drop(fill_in)
del train, test

### Rescaling
Rescaling the input to be between 0 and 1 help the CNN to converge faster and better.
Without this, learning size of weights and thereby gradients will be unbalanced across layers and we would need smaller learning rates, increasing the risk for numerical errors.

In [ ]:
X_train = X_train/255
X_test = X_test/255

### Reshaping
Some frameworks for training NNs, including Keras assume that input is a tensor of higher dimension than 2 (which would be matrix). The input tensor here should be num_examples x height x width x channels. Number of channels is 1 since we have gray-scale pictures, and the -1 means that numpy should figure out how many rows.
Reshaping thus reshapes our n x 28^2 tensor to a n x 28 x 28 x 1 tensor.
Finally we are using the 2D geometri of the data.

In [ ]:
X_train = X_train.values.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28,28,1)

### Exercise (i)
Since X_train and X_test are now numpy arrays, lets take a look at them. We can use indexing and slicing on numpy objects in a similar way we have done on pandas dataframes, however this is easier and does not require loc and iloc.
- Use the .shape attribute of numpy arrays to find out the shape of part of the tensor which hold the first image.
- Remove the last dimension from this part of the tensor and confirm that the result has shape (28,28).

### Label encoding
The targets (or labels as they can be called), have numerical values which seems to indicate that guessing a nearby number is better than guessing a far number. However we only care about guessing right or wrong so that is not the loss function we want. We will therefore apply one-hot encoding to the labels, representing for example the number 3 as \[0,0,3,0,0,0,0,0,0,0\]. This is the expected format of the loss function categorical crossentropy that we will use.

In [ ]:
Y_train = to_categorical(Y_train, num_classes = 10)
Y_test = to_categorical(Y_test, num_classes = 10)

### Neural network architecture

An important part of training a neural network is to design an architecture. We will use a feed-forward
architecture https://en.wikipedia.org/wiki/Feedforward_neural_network with convolutional layers as the main components. A classical approach is to intertwine convolutional layers https://en.wikipedia.org/wiki/Convolutional_neural_network with maxpooling layers and finish off with fully connected layers. Moreover, to prevent overfitting (which could happen very easily with the huge number of variables), dropout layers help a great deal.

A part of the network architecture are so called activation functions which are typically stuck between two layers. Activation functions are simply applied on each value (neuron) before being fed into the next layer.

A popular and powerful activation function is the rectified linear unit, or ReLU. https://en.wikipedia.org/wiki/Rectifier_(neural_networks)

### Convolutional layer

A convolutional layer has several filters which can be seen as small matrixes which determine how to average m-by-m pixels of the image into a single value. In Keras this m is called kernel_size and the number of filters is simply 'filters'. Each filter could learn different things, which is why we need more than 1 filter. For example a filter could detect 45 degree lines , or it could detect upper-right-corners.

What is nice about this is that we don't need to tell the algorithm what filters (or features) are important.

When several convolutional (or other layers) are stacked on top of eachother, higher level features can be learned such as arcs of different curvature, or even body parts or cats or dogs.

In this and the following exercises we will build a multi-layer architecture which works well for MNIST. Credit for the exact architecture and parameter values goes to Yassine Ghouzam, however the idea this type of structure, and many other architectures are well-known in the litterature. See for example https://arxiv.org/pdf/1404.7828.pdf

### Exercise (j)
* Add a first convolutional layer with 32 filters, averaging over 5-by-5 pixels, activation='relu', input_shape=(28,28,1) and padding ='Same'. Run help(Conv2D) and the cell above to get some info.

The setting for padding makes sure that the image doesn't shrink after convolution.

In [ ]:
# Feed-forward network
model = Sequential()

# __fill_in__ #

### Rinse and repeat
Add another identical layer, but do not include input_shape since this is not the first layer.

In [ ]:
## repeat

### Maxpooling
A maxpooling layer divides up the image into k-by-k boxes side by side. Each box gives one node in the layer and the value of that node is simply the maximum of the values in the k-by-k box.

### Exercise (k)
* Add a maxpooling layer with 2-by-2 size.

### Dropout
A Dropout layer randomly sets a fraction `rate` of input units to 0 at each update during training time. The random choice is made independently of both neuron and training example.

### Exercise (l)
* Add a dropout layer with 25% rate.

### Exercise
- Take a moment to think about how the shape of the input tensor changes as it traverses each of the layers we have added so far.
- Write down the sizes you think are correct.
- When you are done, run the next cell.
- Can you figure out why the first layer has 832 parameters and the next one has 25632? You might think the first one has 800 and the next one has 25600.

In [ ]:
model.summary()

### Exercise (m)

- Add another two convolutional layers as before with 64 filters each and size 3-by-3.
- Follow this up by a 2-by-2 maxpooling layer.
- Another 25% dropout.
- Write down what you think the shape of the output is after each layer.
- Take a look at the model summary.

This will amount to 4 new layers.

### Dense layers
Popular CNN+Maxpooling architectures usually finish off with a few so called dense layers, also called fully connected layers. Each neuron in a fully connected layer has an incomming connection from _every_ neuron in the previous layer. This makes the layer very general but also computationally heavier. The generality means that it could capture features with far reaching dependencies such as "curved arc near top - pointy angle approximately N pixel below and M pixels left" -- useful if you want to find the number two.

The Dense layer assumes that the neurons (or equivalently the output) of the previous layer is flattened into a single line, i.e., the shape should be (NONE, d). (The NONE represents the number-of-images dimension which is not known beforehand.)

Luckily there is a layer for reshaping the previous layer, the Flatten layer. You can create Flatten without any configuring parameters.

### Exercise (n)
- Add a Flatten layer.
- Add a Dense layer with 256 neurons and relu activation.
- Add 50% Dropout.
- Add another Dense layer with 10 units and "softmax" activation.
- Rejoice, all layers are in place.

Why does the last layer have 10 output units?

### Softmax explaination
A softmax activation or softmax layer does two things. First it applies the exponential function to each output in the previous layer. Then it rescales each output so that the sum of all outputs is 1.
When the sum of outputs is 1, we can think of the output vector as a probability distribution over a categorical variable. The name softmax comes from the fact that when the exponential function is applied, the maximal value becomes even more dominating so the output vector tends to be close, but not equal to, a vector with a single 1 and the rest of its elements 0.

https://en.wikipedia.org/wiki/Softmax_function

## Optimization and loss
In machine learning the most popular framework for choosing a model from the set of "allowed models" (called hypothesis space), is to first pick some _loss function_, and then solving the optimization problem

pick model that minimizes loss(model,training_data)

In our case the loss will be categorical crossentropy loss. We explain first what the loss is for a single image example where the correct label is n.
Let q_n be the probability our NN outputs for label n. The loss is then -log(q_n).
Thus if the model predicts 0 probability of the correct label (this cannot in fact happen), then the loss is negative infinity. And if the model predicts 1 probability of the correct label, then the loss is 0 which is the best possible value (also doesnt happen).
The loss for the entire training set is then just the total or average loss over all training examples. 

There are many different optimization algorithms for picking the model weights that give the minimal loss, but that is out of the scope of this notebook. We will just try RMSprop.

Links for later reading:

https://en.wikipedia.org/wiki/Cross_entropy

https://en.wikipedia.org/wiki/Gradient_descent

https://en.wikipedia.org/wiki/Stochastic_gradient_descent

https://en.wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp

In [ ]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

## Run training
When training, i.e. optimization is run, the training set is traversed a number of times, each time is called an epoch. During each epoch groups of training examples are fed to the optimizer. Such a group is called a batch.
When a batch is processed, the optimizer tries to slightly tweak the model parameters to get an improved model.

### Exercise (o)
- Use model.fit which is described here: https://keras.io/models/model/, or run help(model.fit).
- Needed parameters are x,y,batch_size, epochs, validation_data. Recommended is also verbose=2.
- Save the result of model.fit in a variable called history.

In [ ]:
# try also with epochs=2
epochs = 1
batch_size = 86

# __fill_in__ #

## The results

In [ ]:
print(history.history)

In [ ]:
# This only makes sense for at least 2 epochs

# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['acc'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_acc'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)